In [1]:
import pandas as pd
import numpy as np

In [2]:
import psycopg2

print("Connecting to the PostgreSQL database...")

Connecting to the PostgreSQL database...


In [3]:
# Establish a connection to the PostgreSQL database
try:
    connection = psycopg2.connect(
        host="db.pqebyefuvarvwvwovchh.supabase.co",
        database="postgres",
        user="postgres",
        password="Hoangthinh@2004"
    )
    cursor = connection.cursor()
    print("Connection successful!")
except Exception as e:
    print(f"An error occurred: {e}")

Connection successful!


In [4]:
query = """
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        """

try:
    cursor.execute(query)
    records = cursor.fetchall()
except (psycopg2.errors.InvalidTextRepresentation, psycopg2.errors.InFailedSqlTransaction) as e:
    print(f"An error occurred: {e}")
    connection.rollback()

    cursor.execute(query)
    records = cursor.fetchall()

df = pd.DataFrame(records, columns=[desc[0] for desc in cursor.description])
df.head(20)

,table_name
0,roles
1,users
2,restaurants
3,signature_foods
4,health_profiles
5,categories
6,tags
7,user_tags
8,restaurant_tags
9,posts


In [ ]:
# Fetch data from the restaurants and related tables
cursor.execute("""
    select b.restaurant_id
        ,a.name as restaurant_name
        ,a.address
        ,c.category_id
        ,d.name as category_name
        ,b.tag_id
        ,c.name as tag_name
    from restaurants a
    inner join restaurant_tags b
        on a.id = b.restaurant_id
    inner join tags c
        on c.id = b.tag_id
    inner join categories d 
        on d.id = c.category_id
""")
records = cursor.fetchall() # Fetch all records from the executed query

# Create a DataFrame from the fetched records
restaurants = pd.DataFrame(records, columns=[desc[0] for desc in cursor.description])
restaurants.head(10)

In [6]:
# Create restaurant profile by aggregating tags

# Concat tag_name seperate by | restaurant_id
restaurants_profile = restaurants.groupby(["restaurant_id", "restaurant_name", "address"])["tag_name"] \
                                 .apply(lambda x: ' | '.join(x)) \
                                 .reset_index(name="restaurantCharacteristics")
restaurants_profile.head(10)

,restaurant_id,restaurant_name,address,restaurantCharacteristics
0,1,Pho Viet Budget,"Hanoi, Vietnam",Budget | Vietnamese | Noodles | Casual | Cozy
1,2,Tokyo Sushi Luxury,"Ho Chi Minh City, Vietnam",Premium | Japanese | Seafood | Luxury | Modern
2,3,Da Nang Vegan Cozy,"Da Nang, Vietnam",MidRange | Vegan | Cozy | Healthy | Themed
3,4,Hanoi Steak House,"Hanoi, Vietnam",MidRange | Vietnamese | Steak | Luxury | Modern
4,5,Saigon Ramen Buffet,"Ho Chi Minh City, Vietnam",Budget | Japanese | Ramen | Buffet | Casual
5,6,Korean BBQ Midrange,"Seoul, Korea",MidRange | Korean | Seafood | Casual | Cozy
6,7,French Fine Dining,"Paris, France",Premium | Korean | Healthy | Luxury | Modern
7,8,Italian Cozy Pasta,"Rome, Italy",MidRange | Korean | Noodles | Cozy | Themed
8,9,Thai Spicy Street Food,"Bangkok, Thailand",Budget | Korean | Spicy | Casual | Themed
9,10,Mexican Themed Cantina,"Mexico City, Mexico",MidRange | Korean | Spicy | Buffet | Themed


In [ ]:
connection.rollback()

In [7]:
# Fetch data from the users and related tables
cursor.execute("""
select b.user_id
      ,a.username
      ,c.category_id
      ,d.name as category_name
      ,b.tag_id
      ,c.name as tag_name
from users a
inner join user_tags b
  on b.user_id = a.id
inner join tags c
    on c.id = b.tag_id
inner join categories d 
    on d.id = c.category_id
""")
records = cursor.fetchall() # Fetch all records from the executed query

# Create a DataFrame from the fetched records
users = pd.DataFrame(records, columns=[desc[0] for desc in cursor.description])
users.head(10)

,user_id,username,category_id,category_name,tag_id,tag_name
0,1,alice,2,Country,10,Vietnamese
1,1,alice,8,Decoration Style,71,Cozy
2,1,alice,1,Price Range,1,Budget
3,2,bob,2,Country,11,Japanese
4,2,bob,4,Restaurant Style,31,Luxury
5,2,bob,3,Food Type,20,Seafood
6,3,charlie,3,Food Type,21,Vegan
7,3,charlie,5,Food Characteristics,41,Healthy
8,3,charlie,1,Price Range,2,MidRange


In [8]:
# create user profile by aggregating tags
# Concat tag_name seperate by | user_id
users_profile = users.groupby(["user_id","username"])["tag_name"] \
                     .apply(lambda x: ' | '.join(x)) \
                     .reset_index(name="userCharacteristics")
users_profile.head(10)

,user_id,username,userCharacteristics
0,1,alice,Vietnamese | Cozy | Budget
1,2,bob,Japanese | Luxury | Seafood
2,3,charlie,Vegan | Healthy | MidRange


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# create user_metrics by choose userID and userCharacteristics
user_metrics = users_profile[["user_id","userCharacteristics"]]

In [ ]:
# create restaurant_metrics by choose restaurantID and restaurantCharacteristics
restaurant_metrics = restaurants_profile[["restaurant_id","restaurantCharacteristics"]]

In [ ]:
# create vectorizer
vectorizer = TfidfVectorizer()

user_vecs = vectorizer.fit_transform(users_profile["userCharacteristics"])
restaurant_vecs = vectorizer.transform(restaurants_profile["restaurantCharacteristics"])

# similarity between userCharacteristics and restaurantCharacteristics
similarity = cosine_similarity(user_vecs, restaurant_vecs)
print(similarity)


In [ ]:
# create final result 
df_result = pd.DataFrame(similarity
                       , index=users_profile["user_id"]
                       , columns=restaurants_profile["restaurant_id"])
df_result.head(10)

In [ ]:
df_long = df_result.reset_index() \
                   .melt(id_vars="user_id", var_name="restaurant_id", value_name="similarity_score")
df_final = df_long.sort_values(by=["user_id", "similarity_score"], ascending=[True, False])
df_final.head(10)

In [10]:
# create pyspark dataframe
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RecommendationSystem").getOrCreate()

Py4JJavaError: An error occurred while calling o46.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChec

In [ ]:
df_res